In [2]:
import sqlite3

In [3]:
# sql 연결을 추가: inmemory 방식
con = sqlite3.connect(':memory:')

In [4]:
# 결과를 가져오는 객체
cur = con.cursor()

### SQL 실행 방법

1. **단일 SQL 실행 (`cur.execute()`)**

   - `cur.execute()`는 하나의 SQL문을 실행하는 방법입니다. 주로 SELECT, INSERT, UPDATE, DELETE 등 단일 쿼리를 실행할 때 사용됩니다.
   
   ```python
   cur.execute("SELECT * FROM table_name")
   ```

2. **여러 SQL 실행 (`cur.executemany()`)**

   - `cur.executemany()`는 동일한 SQL문을 여러 번 실행할 때 사용됩니다. 보통 파라미터를 다르게 하여 여러 번의 쿼리를 실행할 때 유용합니다.
   
   ```python
   query = "INSERT INTO table_name (column1, column2) VALUES (?, ?)"
   data = [(value1, value2), (value3, value4), (value5, value6)]
   cur.executemany(query, data)
   ```

3. **여러 SQL문 실행 (`cur.executescript()`)**

   - `cur.executescript()`는 여러 개의 SQL문을 한 번에 실행할 때 사용됩니다. SQL문이 하나씩 구분된 문자열로 입력되며, 모든 쿼리가 연속해서 실행됩니다.
   
   ```python
   cur.executescript("""
       CREATE TABLE table_name (id INTEGER, name TEXT);
       INSERT INTO table_name (id, name) VALUES (1, 'John');
       INSERT INTO table_name (id, name) VALUES (2, 'Alice');
   """)
   ```

In [5]:
cur.execute("""
    CREATE TABLE TEST (
        ID INTEGER,
        NAME CHAR(1)
    )
""")

In [6]:
cur.execute("INSERT INTO TEST (ID, NAME) VALUES (?, ?);", (1, 'A'))
cur.execute("INSERT INTO TEST (ID, NAME) VALUES (2, 'ABC');")

### SQL 결과 가져오기 방법

1. **1행 가져오기 (`cur.fetchone()`)**

   - `cur.fetchone()`은 쿼리 실행 후, 결과에서 첫 번째 행만 가져옵니다. 한 번 호출할 때마다 한 행씩 반환됩니다.
   
   ```python
   result = cur.fetchone()
   print(result)  # 첫 번째 행을 출력
   ```

2. **N행 가져오기 (`cur.fetchmany(n)`)**

   - `cur.fetchmany(n)`은 쿼리 실행 후, 결과에서 지정한 수만큼의 행을 가져옵니다. `n`에 원하는 행의 수를 입력하여 한 번에 여러 행을 가져올 수 있습니다.
   
   ```python
   results = cur.fetchmany(3)
   print(results)  # 첫 3행을 출력
   ```

3. **전체 행 가져오기 (`cur.fetchall()`)**

   - `cur.fetchall()`은 쿼리 실행 후, 결과에서 모든 행을 가져옵니다. 쿼리 결과가 많을 때는 메모리 사용에 유의해야 합니다.
   
   ```python
   results = cur.fetchall()
   print(results)  # 모든 행을 출력
   ```

In [8]:
cur.execute("SELECT * FROM TEST;")
cur.fetchall()

[(1, 'A'), (2, 'ABC')]

In [9]:
# 접속 해제
con.close()

### db 파일을 생성해서 접속

In [26]:
# 실제로 파일을 만들어서 연결: 확장자 상관없음!
con = sqlite3.connect('coffee.db')
cur = con.cursor()

In [27]:
cur.execute("""
    CREATE TABLE CITY (
        CNO INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL
    );
""")

cur.execute("INSERT INTO CITY (CNO, NAME) VALUES (1, '성북구');")
cur.execute("INSERT INTO CITY (NAME) VALUES ('강북구');")
cur.execute("INSERT INTO CITY VALUES (NULL, '동대문구');")

In [28]:
# 예외발생 case1: not null 
# cur.execute("INSERT INTO CITY VALUES (NULL, NULL);")

In [29]:
# 예외발생 case2: primary key 
# cur.execute("INSERT INTO CITY VALUES (3, '노원구');")

In [30]:
cur.execute("SELECT * FROM CITY;")
cur.fetchall()

[]

In [31]:
# PrimaryKey => auto increment, unique, not null
# 마지막으로 삽입된 행의 ID 가져오기
cur.lastrowid

0

### 데이터베이스 트랜잭션과 커밋

1. **DB에 데이터가 안 들어가 있는 이유**  
   - 데이터베이스에 데이터가 반영되지 않는 이유는 `commit`을 하지 않았기 때문입니다. 데이터베이스는 기본적으로 트랜잭션을 사용하여 데이터의 일관성과 안정성을 보장합니다. 
   
2. **DBMS와 DB**  
   - **DBMS(Database Management System)**: 데이터베이스를 관리하는 시스템으로, 사용자와 데이터베이스 간의 상호작용을 처리합니다.
   - **DB(Database)**: 실제 데이터를 저장하는 장소입니다. DBMS는 데이터베이스를 관리하고 쿼리 등을 실행하여 데이터를 변경합니다.

3. **트랜잭션(Transaction)**  
   - 트랜잭션은 데이터베이스의 상태를 변화시키는 작업을 수행하는 단위입니다. 
   - 트랜잭션은 데이터의 일관성과 무결성을 보장하기 위해 ACID 속성을 따릅니다.
   
   **ACID 속성**  
   - **Atomicity (원자성)**: 트랜잭션 내의 작업은 모두 성공하거나 모두 실패해야 한다는 원칙입니다.
   - **Consistency (일관성)**: 트랜잭션 실행 전후에 데이터가 일관된 상태여야 한다는 원칙입니다.
   - **Isolation (격리성)**: 다른 트랜잭션의 영향을 받지 않도록 보장해야 한다는 원칙입니다.
   - **Durability (내구성)**: 트랜잭션이 성공적으로 커밋된 후에는 그 결과가 영구적으로 저장되어야 한다는 원칙입니다.

4. **데이터 추가/수정(갱신)**  
   - 데이터베이스에 데이터 추가나 수정이 이루어질 때는 트랜잭션을 통해 변화가 발생합니다. 이 변화는 커밋(commit)을 통해 데이터베이스에 반영됩니다.

5. **Commit**  
   - `commit`은 트랜잭션 내에서 이루어진 모든 변경사항을 데이터베이스에 실제로 저장하는 작업입니다. `commit`을 하지 않으면, 트랜잭션은 롤백되어 데이터베이스에 변화가 반영되지 않습니다.


In [32]:
# 커밋하면 db에 반영
con.commit()

### 파라미터 바인딩 방법

1. **`?qmark`**

   - `?qmark`는 SQL 쿼리에서 파라미터를 바인딩할 때 사용하는 자리 표시자입니다. 쿼리문에 `?`를 사용하여 파라미터 값을 동적으로 전달할 수 있습니다.
   - 이 방법은 SQL 인젝션을 방지하는 데 효과적입니다.

   **사용 예시**:
   ```python
   cur.execute("SELECT * FROM users WHERE username = ?", (username,))
   ```

   - 위의 코드에서 `username`은 동적으로 전달되는 값이며, `?` 자리 표시자가 그 자리에 해당 값이 바인딩됩니다.

2. **`:named`**

   - `:named`는 SQL 쿼리에서 명명된 파라미터를 바인딩할 때 사용하는 방법입니다. 파라미터 이름을 `:parameter_name` 형식으로 지정하고, 실행 시 해당 이름에 값을 바인딩합니다.
   - 이 방식은 특히 여러 개의 파라미터를 사용할 때 가독성이 좋고, 코드가 더 명확하게 됩니다.

   **사용 예시**:
   ```python
   cur.execute("SELECT * FROM users WHERE username = :username AND age = :age", {'username': username, 'age': age})
   ```

   - 위의 코드에서 `:username`과 `:age`는 명명된 파라미터로, 각각 `username`과 `age` 값에 대응되는 파라미터가 바인딩됩니다.

In [33]:
cur.execute("INSERT INTO CITY VALUES (NULL, ?);", ('중구',))

In [34]:
cur.execute("SELECT * FROM CITY;")
cur.fetchall()

[(1, '중구')]

In [35]:
cur.execute("INSERT INTO CITY VALUES (NULL, :name);", {'name':'중랑구'})

In [36]:
# 리스트를 이용한 데이터 추가
cur.executemany('''
INSERT INTO CITY (NAME) VALUES (?)
''', (['노원구'], ['강남구'], ['서초구']))

In [38]:
# 딕셔너리를 이용한 데이터 추가
cities = [
    {"name": "Seoul"},
    {"name": "New York"},
    {"name": "London"},
    {"name": "Paris"},
    {"name": "Tokyo"}
]

cur.executemany('''
    INSERT INTO CITY (NAME) VALUES (:name)
''', cities)

### executescript 사용

In [39]:
cur.executescript('''
    DROP TABLE IF EXISTS SUPPLIER;
    CREATE TABLE SUPPLIER(
      SNO INTEGER PRIMARY KEY,
      NAME TEXT,
      CNO INTEGER NOT NULL
    );

    DROP TABLE IF EXISTS PART;
    CREATE TABLE PART(
        PNO INTEGER PRIMARY KEY,
        NAME CHAR(1)
    );
''')

In [40]:
p = [{'n':'아메리카노'}, {'n':'카페라테'}, {'n':'에스프레소'}]
cur.executemany('''
    INSERT INTO PART (NAME) VALUES (:n)
''', p)
# sql문 1개를 4번 반복!

In [41]:
cur.execute("SELECT * FROM PART;")
cur.fetchall()

[(1, '아메리카노'), (2, '카페라테'), (3, '에스프레소')]

### `cursor.execute()` 함수의 동작 과정

`cursor.execute()` 함수는 SQL 쿼리를 실행하는 데 사용되며, 내부적으로 다음과 같은 과정을 거칩니다:

1. **Python 파라미터 2개 넘김 (SQL, 변수)**  
   - `cursor.execute()`는 첫 번째 파라미터로 SQL 쿼리문을 받고, 두 번째 파라미터로는 SQL에 사용할 변수 값들을 전달받습니다. SQL 문과 해당 변수는 파라미터로 제공되며, 이때 파라미터는 동적 값을 입력받아 실행됩니다.

   **사용 예시**:
   ```python
   query = "SELECT * FROM users WHERE username = ? AND age = ?"
   params = ('john_doe', 25)
   cursor.execute(query, params)
   ```

2. **DBMS가 SQL 해석**  
   - DBMS(Database Management System)는 전달받은 SQL 쿼리문을 해석하여 실행 계획을 세웁니다. SQL 쿼리가 올바른 문법을 따르고, 실행될 준비가 되었는지 확인하는 단계입니다.

   - 이 과정에서 DBMS는 쿼리를 최적화하고, 쿼리 실행에 필요한 리소스를 준비합니다. 

3. **DB에서 SQL 적용**  
   - 해석된 SQL 쿼리는 실제로 데이터베이스에 적용됩니다. DBMS는 쿼리 실행을 통해 데이터베이스를 수정하거나, 요청한 데이터를 반환합니다. 이때 SQL문에 포함된 파라미터들은 바인딩되어 쿼리 실행에 반영됩니다.

   **예시:**
   ```python
   cursor.execute("UPDATE users SET age = ? WHERE username = ?", (30, 'john_doe'))
   ```

   - 위 코드에서 `cursor.execute()`는 `UPDATE` SQL 쿼리를 실행하여 `username`이 `'john_doe'`인 사용자의 나이를 30으로 갱신합니다.

In [43]:
# 서초구의 CNO를 가져오는 SQL
cur.execute("SELECT CNO FROM CITY WHERE NAME=?;", ['서초구'])
cur.fetchall()

[(5,)]

In [44]:
# LIMIT 시작위치, 몇개
cur.execute("SELECT CNO FROM CITY WHERE NAME LIKE ? ORDER BY NAME, CNO LIMIT 0, 1;", ['%구'])

In [45]:
cno = cur.fetchone()[0]
cno

4

In [46]:
# 데이터 삽입 (SUPPLIER 테이블)
# SQL 프로시저를 많이 사용함
cur.executemany('''
    INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, ?)
''', [
    ('1호점', cno),
    ('2호점', cno),
    ('3호점', cno)
])

In [49]:
cur.execute('''
    INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, (
            SELECT CNO FROM CITY WHERE NAME LIKE ? 
            ORDER BY NAME, CNO 
            LIMIT 0, 1
    ))
''', ('1호점', '%서초구%'))

In [51]:
cur.execute('select * from supplier;')
cur.fetchall()

[(1, '1호점', 4), (2, '2호점', 4), (3, '3호점', 4), (4, '1호점', 5)]

In [52]:
cur.execute('select name from city where cno>2;')
c = cur.fetchall()

for name in c:
    name = '%'+name[0]+'%'
    params = [{'no':'1호점', 'name':name}, {'no':'2호점', 'name':name}, {'no':'3호점', 'name':name}]

    cur.executemany('''
        INSERT INTO SUPPLIER (NAME, CNO) VALUES (:no, (
                SELECT CNO FROM CITY WHERE NAME LIKE :name
                ORDER BY NAME, CNO 
                LIMIT 0, 1
        ))
    ''', params)

In [53]:
cur.execute('select * from supplier;')
cur.fetchall()

[(1, '1호점', 4),
 (2, '2호점', 4),
 (3, '3호점', 4),
 (4, '1호점', 5),
 (5, '1호점', 3),
 (6, '2호점', 3),
 (7, '3호점', 3),
 (8, '1호점', 4),
 (9, '2호점', 4),
 (10, '3호점', 4),
 (11, '1호점', 5),
 (12, '2호점', 5),
 (13, '3호점', 5),
 (14, '1호점', 6),
 (15, '2호점', 6),
 (16, '3호점', 6),
 (17, '1호점', 7),
 (18, '2호점', 7),
 (19, '3호점', 7),
 (20, '1호점', 8),
 (21, '2호점', 8),
 (22, '3호점', 8),
 (23, '1호점', 9),
 (24, '2호점', 9),
 (25, '3호점', 9),
 (26, '1호점', 10),
 (27, '2호점', 10),
 (28, '3호점', 10),
 (29, '1호점', 6),
 (30, '2호점', 6),
 (31, '3호점', 6),
 (32, '1호점', 7),
 (33, '2호점', 7),
 (34, '3호점', 7),
 (35, '1호점', 8),
 (36, '2호점', 8),
 (37, '3호점', 8),
 (38, '1호점', 9),
 (39, '2호점', 9),
 (40, '3호점', 9),
 (41, '1호점', 10),
 (42, '2호점', 10),
 (43, '3호점', 10)]

In [54]:
cur.execute('select count(pno) from part')
cur.fetchone()[0]

3

In [55]:
cur.execute('''
    create table sells(
        sno integer not null,
        pno integer not null,
        price integer not null default 0
    )
''')

In [56]:
cur.executemany('insert into sells values (?, ?, ?)',
[(1,1,10),
(2,1,10),
(3,1,10),
(5,1,10),
(7,1,10),
(9,1,10),
(11,1,10),
(1,2,15),
(4,2,15),
(6,2,15),
(8,2,15),
(10,2,15),
(12,2,15),
(14,2,15),
(1,3,8),
(18,3,8),
(19,3,8),
(22,4,8),
(1,4,20),
(2,4,20),
(3,4,20)])

In [57]:
cur.execute('select name from city')
cur.fetchall()

[('중구',),
 ('중랑구',),
 ('노원구',),
 ('강남구',),
 ('서초구',),
 ('Seoul',),
 ('New York',),
 ('London',),
 ('Paris',),
 ('Tokyo',),
 ('Seoul',),
 ('New York',),
 ('London',),
 ('Paris',),
 ('Tokyo',)]

In [58]:
cur.execute('insert into city (name) values ("광진구");')

### SQL JOIN

`JOIN`은 두 개 이상의 테이블을 결합하여 하나의 결과 집합을 생성하는 SQL 명령어입니다. 일반적으로 관계형 데이터베이스에서는 여러 테이블 간의 관계를 설정하고, 이를 바탕으로 데이터를 결합하여 원하는 결과를 추출합니다.

#### JOIN의 종류

1. **INNER JOIN**
   - `INNER JOIN`은 두 테이블에서 일치하는 행만 반환합니다. 두 테이블의 공통된 컬럼 값이 일치하는 경우에만 데이터를 가져옵니다.
   
   **사용 예시**:
   ```sql
   SELECT employees.name, departments.name
   FROM employees
   INNER JOIN departments ON employees.department_id = departments.id;
   ```
   - 위 쿼리는 `employees` 테이블과 `departments` 테이블에서 `department_id`와 `id`가 일치하는 행만 결합하여 직원 이름과 부서 이름을 반환합니다.

2. **LEFT JOIN (또는 LEFT OUTER JOIN)**
   - `LEFT JOIN`은 왼쪽 테이블의 모든 행과, 오른쪽 테이블에서 일치하는 행을 결합합니다. 오른쪽 테이블에서 일치하는 값이 없으면, 해당 열은 `NULL`로 채워집니다.
   
   **사용 예시**:
   ```sql
   SELECT employees.name, departments.name
   FROM employees
   LEFT JOIN departments ON employees.department_id = departments.id;
   ```
   - 위 쿼리는 `employees` 테이블의 모든 직원과, 일치하는 부서가 있다면 그 부서 이름도 반환합니다. 부서가 없는 직원의 경우, 부서 이름은 `NULL`로 표시됩니다.

3. **RIGHT JOIN (또는 RIGHT OUTER JOIN)**
   - `RIGHT JOIN`은 오른쪽 테이블의 모든 행과, 왼쪽 테이블에서 일치하는 행을 결합합니다. 왼쪽 테이블에서 일치하는 값이 없으면, 해당 열은 `NULL`로 채워집니다.
   
   **사용 예시**:
   ```sql
   SELECT employees.name, departments.name
   FROM employees
   RIGHT JOIN departments ON employees.department_id = departments.id;
   ```
   - 위 쿼리는 `departments` 테이블의 모든 부서와, 일치하는 직원이 있으면 그 직원 이름도 반환합니다. 직원이 없는 부서의 경우, 직원 이름은 `NULL`로 표시됩니다.

4. **FULL JOIN (또는 FULL OUTER JOIN)**
   - `FULL JOIN`은 양쪽 테이블의 모든 행을 결합합니다. 일치하는 행이 없다면, 결합되지 않은 테이블의 열은 `NULL`로 채워집니다.
   
   **사용 예시**:
   ```sql
   SELECT employees.name, departments.name
   FROM employees
   FULL JOIN departments ON employees.department_id = departments.id;
   ```
   - 위 쿼리는 `employees` 테이블과 `departments` 테이블의 모든 행을 반환합니다. 일치하는 데이터가 없으면, 해당 컬럼에 `NULL`이 표시됩니다.

5. **CROSS JOIN**
   - `CROSS JOIN`은 두 테이블의 모든 조합을 반환합니다. 즉, 첫 번째 테이블의 각 행이 두 번째 테이블의 모든 행과 결합됩니다. 이 경우 결과의 행 수는 두 테이블의 행 수의 곱이 됩니다.
   
   **사용 예시**:
   ```sql
   SELECT employees.name, departments.name
   FROM employees
   CROSS JOIN departments;
   ```
   - 위 쿼리는 `employees` 테이블과 `departments` 테이블의 모든 가능한 조합을 반환합니다. 즉, 모든 직원과 모든 부서가 결합된 결과를 출력합니다.

In [59]:
cur.execute('''
    select city.name, supplier.name from city
    inner join supplier
    on city.cno=supplier.cno
''')
print(len(cur.fetchall()))

cur.execute('''
    select city.name, supplier.name from city
    left join supplier
    on city.cno=supplier.cno
''')
print(len(cur.fetchall()))

43
51


In [60]:
cur.execute('''
    select city.name, count(supplier.name) from city
    left join supplier
    on city.cno=supplier.cno
    group by city.cno
    order by city.name asc
''')

cur.fetchall()

[('London', 6),
 ('London', 0),
 ('New York', 6),
 ('New York', 0),
 ('Paris', 6),
 ('Paris', 0),
 ('Seoul', 6),
 ('Seoul', 0),
 ('Tokyo', 6),
 ('Tokyo', 0),
 ('강남구', 6),
 ('광진구', 0),
 ('노원구', 3),
 ('서초구', 4),
 ('중구', 0),
 ('중랑구', 0)]

In [61]:
cur.execute('''
    select city.name, supplier.name, part.name, count(part.name), sum(sells.price) from city
    left join supplier
    on city.cno = supplier.cno
    inner join sells
    on supplier.sno=sells.sno
    inner join part
    on part.pno=sells.pno
    group by city.name, supplier.name
    order by city.cno asc
''')

cur.fetchall()

[('노원구', '1호점', '아메리카노', 1, 10),
 ('노원구', '2호점', '카페라테', 1, 15),
 ('노원구', '3호점', '아메리카노', 1, 10),
 ('강남구', '1호점', '아메리카노', 4, 48),
 ('강남구', '2호점', '아메리카노', 2, 20),
 ('강남구', '3호점', '아메리카노', 2, 25),
 ('서초구', '1호점', '아메리카노', 2, 25),
 ('서초구', '2호점', '카페라테', 1, 15),
 ('Seoul', '1호점', '카페라테', 1, 15),
 ('New York', '2호점', '에스프레소', 1, 8),
 ('New York', '3호점', '에스프레소', 1, 8)]